# 난 스케치를 할 테니 너는 채색을 하거라
-----
### 프로젝트 - Segmentation map으로 도로 이미지 만들기

![](https://aiffelstaticprd.blob.core.windows.net/media/images/p2p_result_seg.max-800x600.png)

이번 프로젝트는 위와 같은 데이터셋을 이용해 Pix2Pix를 학습시켜보는 것입니다. 데이터셋은 아래에서 다운로드할 수 있습니다.
1000개의 학습용 이미지 및 5개의 평가 이미지를 포함합니다.

>* [cityscapes.zip](https://aiffelstaticprd.blob.core.windows.net/media/documents/cityscapes.zip)

프로젝트 시작에 어려움이 있다면 아래 튜토리얼을 참고하시면서 이번 프로젝트를 수행해봅시다.
(아래 튜토리얼은 Pix2Pix 구조를 Functional API를 이용해 구현하기 때문에 이번에 진행한 Subclassing 방법을 이용한 모델 구현과 비교하면서 구현 방법에 대해서도 공부해 보시길 추천드립니다)
> * [Tensorflow Pix2Pix Tutorial](https://www.tensorflow.org/tutorials/generative/pix2pix?hl=ko)

## 목표
>1. 조건을 부여하여 생성 모델을 다루는 방법에 대해 이해합니다.
>2. cGAN 및 Pix2Pix의 구조와 학습 방법을 이해하고 잘 활용합니다.
>3. CNN 기반의 모델을 구현하는데 자신감을 갖습니다.

## 프로젝트 수행
-----
프로젝트를 진행하면서 필수로 수행해야 할 사항은 다음과 같습니다.

>1. 데이터에 한 가지 이상의 **augmentation 방법을 적용**하여 학습해주세요.
(어떠한 방법을 사용했는지 적어주세요.)
>2. 이전에 구현했던 두 개의 Generator 중 Encoder와 Decoder간에 skip connection이 있는 **U-Net Generator를 사용**해주세요.
>3. 모델 학습 후, 학습된 Generator를 이용해 테스트합니다. 테스트 데이터는 다운받았던 "val" 폴더 내 이미지를 사용해주세요.
>4. 1개 이상의 이미지에 대해 테스트 과정을 거친 후 그 결과를 **스케치, 생성된 사진, 실제 사진 순서로 나란히 시각화**해 주세요.
>5. 모델을 충분히 학습하기에 시간이 부족할 수 있습니다. 적어도 10 epoch 이상 학습하며 **중간 손실 값에 대한 로그**를 남겨주세요. 좋은 결과를 얻기 위해선 긴 학습 시간이 필요하므로 테스트 결과는 만족스럽지 않아도 괜찮습니다.

## 용어정리
---

**✓ Conditional Generative Adversarial Nets (cGAN)**이란?
>* 원하는 종류의 이미지를 생성하고자 할 때 GAN이 가진 생성 과정의 불편함을 해소하여, 원하는 종류의 이미지를 생성할 수 있도록 고안된 방법
>* cGAN 목적함수에서 바뀐 부분은 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>y</mi>
</math>라는 정보가 함께 입력된다는 것
> ![](https://aiffelstaticprd.blob.core.windows.net/media/images/cgan_img.max-800x600.png)
>* **Generator**이란?
>> * 노이즈 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>z</mi>
</math>(파란색)와 추가 정보 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>y</mi>
</math>(녹색)을 함께 입력받아 Generator 내부에서 결합되어 representation(검정색)으로 변환되며 가짜 데이터<math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>G</mi>
  <mo stretchy="false">(</mo>
  <mi>z</mi>
  <mo>&#x2223;</mo>
  <mi>y</mi>
  <mo stretchy="false">)</mo>
</math>를 생성
>>* MNIST나 CIFAR-10 등의 데이터셋에 대해 학습시키는 경우 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>y</mi>
</math>는 레이블 정보이며, 일반적으로 one-hot 벡터를 입력으로 넣음
>* **Discriminator**이란?
>>* 실제 데이터 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>x</mi>
</math>와 Generator가 생성한 가짜 데이터 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>G</mi>
  <mo stretchy="false">(</mo>
  <mi>z</mi>
  <mo>&#x2223;</mo>
  <mi>y</mi>
  <mo stretchy="false">)</mo>
</math>를 각각 입력받으며, 마찬가지로 정보가 각각 함께 입력되어 진짜와 가짜를 식별
>>* MNIST나 CIFAR-10 등의 데이터셋에 대해 학습시키는 경우 실제 데이터 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>x</mi>
</math>와 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>y</mi>
</math>는 알맞은 한 쌍("7"이라 쓰인 이미지의 경우 레이블도 7)을 이뤄야 함
>>* 마찬가지로 Generator에 입력된 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>y</mi>
</math>와 Discriminator에 입력되는 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>y</mi>
</math>는 동일한 레이블을 나타내야 함

**✓ 데이터 로드**

1) wget으로 데이터 다운로드
> $ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/sketch2pokemon.zip

> $ mv sketch2pokemon.zip ~/aiffel/conditional_generation

> $ cd ~/aiffel/conditional_generation && unzip sketch2pokemon.zip

**TIPS]** (❗주의) **커널 메모리를 체크해 주세요!**
이전 모델이 메모리에 로드되어 있는 상태에서 pix2pix 모델을 함께 학습하면 메모리 오류가 발생할 수도 있습니다. 메모리가 충분하지 않다면 이 시점에 커널 재시작 후 진행할 것을 권합니다.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
data_path = os.getenv('HOME')+'/aiffel/conditional_generation/pokemon_pix2pix_dataset/train/'
print("number of train examples :", len(os.listdir(data_path)))

In [ ]:
# 각각의 이미지들이 어떻게 생겼는지 확인
# 학습용 데이터셋에서 임의로 6장을 선택하여 시각화
plt.figure(figsize=(20,15))
for i in range(1, 7):
    f = data_path + os.listdir(data_path)[np.random.randint(800)]
    img = cv2.imread(f, cv2.IMREAD_COLOR)
    plt.subplot(3,2,i)
    plt.imshow(img)

In [ ]:
# 이미지 크기를 확인
f = data_path + os.listdir(data_path)[0]
img = cv2.imread(f, cv2.IMREAD_COLOR)

print(img.shape)

In [ ]:
def normalize(x):
    x = tf.cast(x, tf.float32)
    return (x/127.5) - 1

def denormalize(x):
    x = (x+1)*127.5
    x = x.numpy()
    return x.astype(np.uint8)

def load_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, 3)
    
    w = tf.shape(img)[1] // 2
    sketch = img[:, :w, :] 
    sketch = tf.cast(sketch, tf.float32)
    colored = img[:, w:, :] 
    colored = tf.cast(colored, tf.float32)
    return normalize(sketch), normalize(colored)

f = data_path + os.listdir(data_path)[1]
sketch, colored = load_img(f)

plt.figure(figsize=(10,7))
plt.subplot(1,2,1); plt.imshow(denormalize(sketch))
plt.subplot(1,2,2); plt.imshow(denormalize(colored))

In [ ]:
from tensorflow import image
from tensorflow.keras.preprocessing.image import random_rotation

@tf.function() # 빠른 텐서플로 연산을 위해 @tf.function()을 사용합니다. 
def apply_augmentation(sketch, colored):
    stacked = tf.concat([sketch, colored], axis=-1)
    
    _pad = tf.constant([[30,30],[30,30],[0,0]])
    if tf.random.uniform(()) < .5:
        padded = tf.pad(stacked, _pad, "REFLECT")
    else:
        padded = tf.pad(stacked, _pad, "CONSTANT", constant_values=1.)

    out = image.random_crop(padded, size=[256, 256, 6])
    
    out = image.random_flip_left_right(out)
    out = image.random_flip_up_down(out)
    
    if tf.random.uniform(()) < .5:
        degree = tf.random.uniform([], minval=1, maxval=4, dtype=tf.int32)
        out = image.rot90(out, k=degree)
    
    return out[...,:3], out[...,3:]   

print("✅")

# 루브릭 평가 기준
---
아래의 기준을 바탕으로 프로젝트를 평가합니다.

번호|평가문항|상세기준
---|---|---
1 |pix2pix 모델 학습을 위해 필요한 데이터셋을 적절히 구축하였다.|데이터 분석 과정 및 한 가지 이상의 augmentation을 포함한 데이터셋 구축 과정이 체계적으로 제시되었다.
2 |pix2pix 모델을 구현하여 성공적으로 학습 과정을 진행하였다.|U-Net generator, discriminator 모델 구현이 완료되어 train_step이 안정적으로 진행됨을 확인하였다.
3 |학습 과정 및 테스트에 대한 시각화 결과를 제출하였다.|10 epoch 이상의 학습을 진행한 후 최종 테스트 결과에서 진행한 epoch 수에 걸맞은 정도의 품질을 확인하였다.

# 회고
---